# Experiment with Bezier curve fitting

This is a Python prototype that might be sufficient. If not, I will port it to C++ and create Python bindings (which I really want to do!).

First, I will experiment w the IAM OnDB Dataset (untransformed sample) and then later with an example obtained from Xournal++.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from src.data.online_handwriting_datasets import IAM_OnDB_Dataset

## Helper functions

These functions are experiments for now and will eventually be moved into the `src/` module.

In [3]:
# TODO

## Load example data

Example data here refers to a single sample that I can experiment with to fit to Bezier curves.

In [ ]:
PATH = Path('../data/datasets/IAM-OnDB') # Needs to be parameterised

ds = IAM_OnDB_Dataset(path=PATH, transform=None, limit=100)

Select a particular sample to focus on now:

In [5]:
i_selected = 0

sample = ds[i_selected]

Plot the sample:

In [ ]:
plt.figure()
plt.scatter(sample['x'], sample['y'], c=sample['stroke_nr'], s=0.8, cmap='tab20')
plt.title(sample['label'])
plt.xlabel('x')
plt.ylabel('y')
plt.gca().set_aspect('equal')
plt.show()

## Optimise data structure for better experimentation

First, I load the sample into `pandas`. Then, I groupby the sample w.r.t. `stroke_nr` and store the result in a simple list for now.

In [7]:
df = pd.DataFrame.from_dict({
    'x': sample['x'],
    'y': sample['y'],
    't': sample['t'],
    'stroke_nr': sample['stroke_nr'],
})

list_of_strokes = []

for stroke_nr, df_grouped in df.groupby('stroke_nr'):
    list_of_strokes.append({
        'x': df_grouped['x'],
        'y': df_grouped['y'],
        't': df_grouped['t'],
    })

## Perform a simple 3rd order polynomial fit

This is essentially a Bezier curve as those are 3rd order polynomials.

First, just focus on $x$ and $y$. Later, check out $t$ as fit target.

In [ ]:
plt.figure()

for i_stroke in range(len(list_of_strokes)):
    
    x = list_of_strokes[i_stroke]['x']
    y = list_of_strokes[i_stroke]['y']
    t = list_of_strokes[i_stroke]['t']
    t_normalised = (t - t.min()) / (t.max() - t.min())
    z_x = np.polyfit(x=t_normalised, y=x, deg=3)
    z_y = np.polyfit(x=t_normalised, y=y, deg=3)
    p_x = np.poly1d(z_x)
    p_y = np.poly1d(z_y)

    # TODO: Can be enabled but it gets messy
    # plt.figure()
    # plt.scatter(t_normalised, x, c='green', label='raw x')
    # plt.scatter(t_normalised, y, c='black', label='raw y')
    # plt.plot(t_normalised, p_x(t_normalised), c='blue', label='fit x')
    # plt.plot(t_normalised, p_y(t_normalised), c='red', label='fit y')
    # plt.xlabel('t_normalised')
    # plt.legend()
    # plt.show()

    plt.scatter(x, y, s=0.8, c='black', label='raw' if i_stroke == 0 else None)
    plt.scatter(p_x(t_normalised), p_y(t_normalised), s=0.8, c='red', label='fitted' if i_stroke == 0 else None)

plt.xlabel('x')
plt.ylabel('y')
plt.gca().set_aspect('equal')
plt.legend()
plt.show()

## Understand [Carbune2020] fitting procedure

I follow the procedure explained in section 2.1.2 of [Carbune2020].

First, normalise size of entire ink such that y values are within [0, 1]:

In [ ]:
df['y_normalised'] = df['y'] - df['y'].min()
scale_factor = df['y_normalised'].max()
df['y_normalised'] = df['y_normalised'] / scale_factor
df['x_normalised'] = df['x'] / scale_factor

plt.figure()
plt.scatter(df['x_normalised'], df['y_normalised'], s=0.3)
plt.title('normalisation debugging')
plt.gca().set_aspect('equal')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [29]:
df['t_normalised'] = np.nan

In [ ]:
for stroke_nr, df_grouped in df.groupby('stroke_nr'):
    print(stroke_nr)

In [ ]:
df_grouped

In [ ]:
df['y_normalised'].max()

I am unclear what they mean w/ "We alternate between minimizing the SSE in Eq. (3) and finding the corresponding points si , until convergence.".

This is my understanding:

In [37]:
# See here: https://mermaid.js.org/ecosystem/tutorials.html#jupyter-integration-with-mermaid-js

import base64
from IPython.display import Image, display
import matplotlib.pyplot as plt

def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

In [ ]:
mm("""
graph LR;
    A[Knowing s_i] --> B[Compute alpha, beta, gamma];
    B --> C[Compute fitted x, y, t];
    C --> D[Update s_i w Newton rule];
    D --> A;
    E[Pick initial s_i] --> A;
    C --> F[Compute SSE];
    G[HOW TO MIN SSE?!?!]
""")

How to constrain s_i in [0, 1]?